# Config

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "your_openai_key_here"
os.environ["GEMINI_API_KEY"] = "your_gemini_key_here"
os.environ["GOOGLE_API_KEY"] = "your_google_key_here"
os.environ["DEEPSEEK_API_KEY"] = "your_deepseek_key_here"


import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST.json",
    filter_dict={
        "model": ["gpt-4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    },
)

llm_config = {
    "timeout": 60,
    "cache_seed": 42,
    "config_list": config_list,
    "temperature": 0,
}


# Autogen Logging

In [ ]:
import re
import urllib.request

def parse_topic_file(path):
    with open(path, "r", encoding="utf-8") as f:
        content = f.read()
    topics = re.findall(r"<topic num='(\d+)'>\s*<desc>(.*?)</desc>", content, re.DOTALL)
    return [(int(num), desc.strip()) for num, desc in topics]

def get_public_ip():
    try:
        return urllib.request.urlopen("https://api.ipify.org").read().decode("utf-8")
    except Exception:
        return "unknown"


In [ ]:
import asyncio
import nest_asyncio
import logging
import datetime
import socket
import time
import subprocess
from urllib.parse import urlparse

from autogen_agentchat.ui import Console
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.models.ollama import OllamaChatCompletionClient
from autogen_ext.agents.web_surfer import MultimodalWebSurfer

nest_asyncio.apply()

isp_logger = logging.getLogger("isp_logger")
isp_logger.setLevel(logging.INFO)
isp_logger.propagate = False


def set_isp_logfile(filename):
    for h in isp_logger.handlers:
        isp_logger.removeHandler(h)
    handler = logging.FileHandler(filename, mode="w")
    handler.setFormatter(logging.Formatter("%(levelname)s | %(message)s"))
    isp_logger.addHandler(handler)

def start_tcpdump(topic_num):
    return subprocess.Popen([
        "sudo", "tcpdump", "-i", "any",
        "-w", f"autogen_log/full_traffic_{topic_num}.pcap"
    ])
    

class LoggingOpenAIClient(OpenAIChatCompletionClient):
    async def acompletion(self, *args, **kwargs):
        messages = kwargs.get("messages") or (args[0] if args else [])
        timestamp = datetime.datetime.utcnow().isoformat()
        domain = "api.openai.com"
        ip = socket.gethostbyname(domain)
        method = "POST"
        try:
            payload_size = len(str(messages).encode())
        except Exception:
            payload_size = "unknown"

        isp_logger.info(f"[ISP] {timestamp} | [API REQUEST] Method: {method} | Domain: {domain} | IP: {ip} | URL: https://api.openai.com/v1/chat/completions | Size: {payload_size}")
        return await super().acompletion(*args, **kwargs)


class LoggingWebSurfer(MultimodalWebSurfer):

    async def _lazy_init(self):
        await super()._lazy_init()

        original_visit_page = self._playwright_controller.visit_page
        original_on_new_page = self._playwright_controller.on_new_page

        async def attach_logging_to_page(page):
            async def log_request(route, request):
                url = request.url
                domain = urlparse(url).netloc or "unknown"
                timestamp = datetime.datetime.utcnow().isoformat()
                try:
                    ip = socket.gethostbyname(domain)
                except Exception:
                    ip = "unknown"
                method = request.method
                size = request.headers.get("content-length", "unknown")
                
                # if "bing.com" not in domain:
                isp_logger.info(f"[ISP] {timestamp} | Domain: {domain} | IP: {ip} | Size: {size} | [REQUEST] Method: {method} |  URL: {url} ")

                await route.continue_()
    
            def log_response(response):
                async def handle():
                    timestamp = datetime.datetime.utcnow().isoformat()
                    try:
                        body = await response.body()
                        size = len(body)
                    except Exception:
                        size = "unknown"
                    status = response.status
                    resp_url = response.url
                    domain = urlparse(resp_url).netloc or "unknown"
                    try:
                        ip = socket.gethostbyname(domain)
                    except Exception:
                        ip = "unknown"

                    # if "bing.com" not in domain:
                    isp_logger.info(f"[ISP] {timestamp} | Domain: {domain} | IP: {ip} | Size: {size} | [RESPONSE] Status: {status} |  URL: {resp_url} ")
                asyncio.create_task(handle())
    
            await page.route("**/*", log_request)
            page.on("response", log_response)

            page.on("framenavigated", lambda frame: isp_logger.info(
                f"[ISP] {datetime.datetime.utcnow().isoformat()} | [NAVIGATED] | Domain: {urlparse(frame.url).netloc} | URL: {frame.url}"
            ))


        async def logging_visit_page(page, url, *args, **kwargs):
            await attach_logging_to_page(page)
            page.context.on("page", lambda p: asyncio.create_task(attach_logging_to_page(p)))
            return await original_visit_page(page, url, *args, **kwargs)
    
        async def logging_on_new_page(page):
            await original_on_new_page(page)
            await attach_logging_to_page(page)
    
        self._playwright_controller.visit_page = logging_visit_page
        self._playwright_controller.on_new_page = logging_on_new_page
    

In [ ]:
from autogen import GroupChat

async def run_all_tasks():

    
    log_file = f"autogen_log/rewritten/gpt4o-DD16-1.txt"

    logger = set_isp_logfile(log_file)

    assistant = AssistantAgent(
        name="LLM_Assistant",
        system_message=(
            "You are part of a web browsing team.\n"
            "Your role is to collaborate with the MultimodalWebSurfer.\n"
            "You must not simulate browsing or guess what might be on websites.\n"
            "Instead, wait for the MultimodalWebSurfer to visit web pages and return content before you respond.\n"
            "Only use information the surfer provides. Do not use prior knowledge or make assumptions.\n"
            "Summarize findings only after the surfer has visited at least 5 pages.\n"
            "Conclude with 'Done with task.' once your summary is complete."
        ),

        model_client=LoggingOpenAIClient(model="gpt-4o")
        # model_client=LoggingOpenAIClient(model="gemini-2.0-flash")
        # model_client=OllamaChatCompletionClient(model="llama3:8b")
    )

    web_surfer_agent = LoggingWebSurfer(
        name="MultimodalWebSurfer",
        model_client=OpenAIChatCompletionClient(model="gpt-4o-2024-08-06"),
        # model_client=OpenAIChatCompletionClient(model="gemini-2.0-flash"),
        # model_client=OllamaChatCompletionClient(model="llama3:8b")
    )

    termination = MaxMessageTermination(100) | TextMentionTermination("Done with task.", sources="MultimodalWebSurfer") \
                                        | TextMentionTermination("ou're welcome") \
                                        | TextMentionTermination("here's anything else") \
                                        | TextMentionTermination("need more assistance") 

    agent_team = RoundRobinGroupChat(
        [assistant, web_surfer_agent],
        termination_condition=termination
    )
    
    full_prompt = f"""**Instructions for Researcher**

**Research Task:**  
I am seeking a detailed report on the U.S. military’s deployment to West Africa to support the response to the Ebola epidemic, with a focus on the following aspects:

1. **Key Leaders:**  
   - Identify field-grade officers (majors, lieutenant colonels, colonels, or equivalent ranks) and senior NCOs from all four U.S. military branches (Army, Navy, Air Force, Marine Corps) who were in direct command of units deployed to West Africa for Ebola support operations.  
   - For each leader, provide:
     - Name
     - Rank and Service
     - Specific position/title during the deployment
     - Unit commanded or managed
     - Location of deployment (country/site if possible)
   - Exclude U.S. military leaders who were not deployed to Africa and any non-U.S. or civilian aid worker leadership.
   
2. **Mission Description:**  
   - Describe the official mission of the deployed units.  
   - Clarify the scope of activities (e.g., facilities construction, logistics, telecommunications, road building, etc.), with an explicit emphasis that military personnel were not involved in direct patient care or treatment of Ebola victims.
   - Quote or summarize official mission statements or orders where available.
   
3. **Personnel Safety Protocols:**  
   - Document the official protocols and procedures enforced to safeguard deployed U.S. personnel against Ebola exposure.
   - Include pre-deployment, in-theater, and post-deployment protocols, specifically including mandatory quarantine or monitoring upon return to home bases.
   - Note any service-specific variations in safety procedures.
   - Reference official military or Defense Department sources describing these protocols.
   
4. **Scope:**  
   - Only include information about U.S. military units that deployed to Africa for the Ebola epidemic response.
   - Exclude non-U.S. military and civilian organization details.
   - Exclude U.S. military units that did not physically deploy to Africa.
   
5. **Table Requirement:**  
   - Present a table summarizing the key leaders, ranks, branches, units, deployment locations, and roles.
   
6. **Expected Output Format:**  
   - Provide a structured report including the following sections and headers:
     1. Executive Summary
     2. Key Leaders Table
     3. Mission Overview
     4. Safety Protocols for Personnel
     5. References
     
7. **Preferred Sources:**  
   - Use official or primary sources where possible, such as:
     - U.S. Department of Defense press releases and reports
     - Official statements from U.S. Africa Command, U.S. Army Africa, and other service branches
     - Public documentation from service component commands (e.g., www.defense.gov, www.army.mil, www.africom.mil)
   - Avoid sources focused on non-U.S. units or civilian aid efforts.
   - Reference only materials in English unless otherwise instructed.
   
8. **Open-Ended Attributes:**  
   - If the precise names/titles of some field-grade officers or senior NCOs cannot be identified in open sources, note this explicitly and provide the highest available command-level detail.
   
**Please ensure the report is well-organized, directly responsive to the parameters above, and cites all sources used. Tables, headers, and structured format are required.**
"""
# To complete the task, you must:
# 1. Begin with a Bing search.
# 2. Click links or manually visit at least 5 different pages—not just summarize search results or rely on prior knowledge.
# 3. Visit and extract information from at least 5 different websites by clicking links.
# 4. Only use information found on pages you actually opened.
# 5. Once you have gathered concrete details from at least 5 different pages, write a brief summary and say: "Done with task."

# Do not infer or assume anything unless it is directly stated on a visited webpage.
# """

    stream = agent_team.run_stream(task=full_prompt)
    await Console(stream, output_stats=True)
    await web_surfer_agent.close()

    # tcpdump_proc.terminate()
    # tcpdump_proc.wait()

    for handler in isp_logger.handlers:
        handler.flush()
    time.sleep(1)


# Run the batch
asyncio.run(run_all_tasks())


In [ ]:
from autogen import GroupChat

# def parse_topic_file(filename):
#     with open(filename, "r") as f:
#         lines = [line.strip() for line in f if line.strip()]
#     return [(i + 1, line) for i, line in enumerate(lines)]

async def run_all_tasks():
    topics = parse_topic_file("topictext-890.txt")
    # topics = parse_topic_file("topictext-5prompt-autogen.txt")

    for topic_num, task_desc in topics:
        log_file = f"autogen_log/gemini-2/agent_traffic_{topic_num}.txt"

        if topic_num not in [2, 4, 22, 29, 34, 36, 41, 42, 51, 57]:
            continue
        if os.path.exists(log_file) and os.path.getsize(log_file) > 0:
            print(f"Skipping topic {topic_num}: already completed.")
            continue

        print(f"Running topic {topic_num}")
        
        # log_file = f"rerun/autogen_agent_traffic_{topic_num}.txt"
        logger = set_isp_logfile(log_file)

        assistant = AssistantAgent(
            name="LLM_Assistant",
            system_message=(
                "You are part of a web browsing team.\n"
                "Your role is to collaborate with the MultimodalWebSurfer.\n"
                "You must not simulate browsing or guess what might be on websites.\n"
                "Instead, wait for the MultimodalWebSurfer to visit web pages and return content before you respond.\n"
                "Only use information the surfer provides. Do not use prior knowledge or make assumptions.\n"
                "Summarize findings only after the surfer has visited at least 5 pages.\n"
                "Conclude with 'Done with task.' once your summary is complete."
            ),

            # model_client=LoggingOpenAIClient(model="gpt-4o")
            model_client=LoggingOpenAIClient(model="gemini-2.0-flash")
            # model_client=OllamaChatCompletionClient(model="llama3:8b")
        )

        web_surfer_agent = LoggingWebSurfer(
            name="MultimodalWebSurfer",
            # model_client=OpenAIChatCompletionClient(model="gpt-4o-2024-08-06"),
            model_client=OpenAIChatCompletionClient(model="gemini-2.0-flash"),
            # model_client=OllamaChatCompletionClient(model="llama3:8b")
        )
    
        termination = MaxMessageTermination(100) | TextMentionTermination("Done with task.", sources="MultimodalWebSurfer") \
                                            | TextMentionTermination("ou're welcome") \
                                            | TextMentionTermination("here's anything else") \
                                            | TextMentionTermination("need more assistance") 

        agent_team = RoundRobinGroupChat(
            [assistant, web_surfer_agent],
            termination_condition=termination
        )
        
        full_prompt = f"""{task_desc.strip()}

Before concluding or summarizing, please collaborate with the MultimodalWebSurfer to explore relevant websites and gather concrete information.

To complete the task, you must:
1. Begin with a Bing search.
2. Click links or manually visit at least 5 different pages—not just summarize search results or rely on prior knowledge.
3. Visit and extract information from at least 5 different websites by clicking links.
4. Only use information found on pages you actually opened.
5. Once you have gathered concrete details from at least 5 different pages, write a brief summary and say: "Done with task."

Do not infer or assume anything unless it is directly stated on a visited webpage.
"""

        stream = agent_team.run_stream(task=full_prompt)
        await Console(stream, output_stats=True)
        await web_surfer_agent.close()

        for handler in isp_logger.handlers:
            handler.flush()
        time.sleep(1)


# Run the batch
asyncio.run(run_all_tasks())


# Filter traffic

In [ ]:
import os
import re
import json

# Directory and file config
# log_dir = "autogen_log"
log_dir = "rerun"
log_prefix = "autogen_agent_traffic_"
log_suffix = ".txt"
# num_files = 60
num_files = 20

# Keywords that indicate noisy, non-user-initiated domains
NOISY_DOMAIN_KEYWORDS = [
    "doubleclick", "hubspot", "adspsp", "onetrust.", "googlesyndication", "adsrvr", "adnxs", "adsafeprotected", "pubmatic", "rubiconproject", "msn", "bing", "googlevideo.", "googletagmanager.", "adform", "salesforceliveagent",
    "mdn", "admaster", "criteo", "rtb", "yieldmo", "taboola", "outbrain", "quantserve", "insightexpress",
    "adservice", "amazon-adsystem", "aps.amazon", "adtrafficquality.google", "ads.", "advertising",
    "banner", "cookie", "ezodn", "ezoic", "adlightning", "liadm.com", "media.net", "pm-serv.co",
    "rqtrk.eu", "mgid", "adblocker", "admanmedia", "hotjar.", "clickguard", "contextweb", "doubleverify",
    "analytics", "google-analytics", "facebook", "twitter", "linkedin", "scorecardresearch", "pixel",
    "tracking", "track", "metrics.", "sentry", "clarity.ms", "fullstory", "crazyegg", "segment",
    "cdn", "fonts", "gstatic", "cloudfront", "cloudflare", "moatads", "optimizely", "js.", "api.",
    "virtualearth", "apis.", "wp.com", "hlx.page", "vo.msecnd.net", "mpio.io",
    "amazonaws.com", "objectstorage", "azureedge.net", "fastly.net", "cdn.jsdelivr.net", "akamaihd.net",
    "qualified", "recombee", "leadsrx", "crwdcntrl", "typekit.", 
    "imasdk", "translate.google", "calendar.google", "accounts.google", "googleapis", "cse.google.com",
    "fundingchoicesmessages.google.com", "google.", "bidsxchange", "avplayer", "usercontent.", "css.", "scripts.",
    "privacymanager.io", "foresee.com", "qualaroo", "trustarc", "truste", "gatekeeperconsent",
    "givebutter", "churnkey", "fundraiseup", "mailerlite", "marketo", "pardot", "hsforms", "paperform",
    "activedemand", "activehosted", "convertkit", "script.", "googletag", "widget",
    "qualtrics", "monsido", "igodigital", "drivetheweb", "addtoany", "liveperson.", 
    "squarespace", "cloudinary", "wistia", "forbesimg", "imageio.forbes.com", "ytimg", "open.video",
    "humix", "recommendation.forbes.com", "analytics.", "google.", "tiktok", "server.", 
    "amazonaws.", "api-domain-compado.com", "xapstream", "adroll", "-app", "3lift.", "beacon",
    "pinterest", "pinimg.com", "quora", "instagram", "yimg.com", "recaptcha", "sharethis", "highperformanceformat.com", "fontawesome.", ".content", "ad.gt", "assets-", 
    "jwplayer", "assets.", "jquery.", "static.", "asset-", "app-", "app.", "js-", "widget.", "captcha", "permutive.app", "youtube.", "lytics", "adthrive.", "wikimedia", "doubleverify",
]

def is_noisy(domain):
    domain = domain.lower()
    return any(keyword in domain for keyword in NOISY_DOMAIN_KEYWORDS)

def extract_size(line):
    match = re.search(r"Size:\s*(\d+|unknown)", line)
    if match:
        size_str = match.group(1)
        return int(size_str) if size_str.isdigit() else None
    return None


# Output dictionary
filtered_domain_seqs = {}

for i in range(1, num_files + 1):
    filename = f"{log_prefix}{i}{log_suffix}"
    path = os.path.join(log_dir, filename)

    valid_domains = []
    seen = set()

    with open(path, "r") as f:
        for line in f:
            if "[ISP]" in line and "Domain:" in line:
                size = extract_size(line)
                if size is None or size < 7000:
                    continue  # skip if size unknown or too small

                domain_match = re.search(r"Domain:\s*([\w\.-]+)", line)
                if domain_match:
                    domain = domain_match.group(1).lower()
                    if not is_noisy(domain) and domain not in seen:
                        valid_domains.append(domain)
                        seen.add(domain)

    filtered_domain_seqs[f"prompt_{i}"] = valid_domains

# Save output
with open("rerun/autogen_filtered_domain_sequences.json", "w") as f:
    json.dump(filtered_domain_seqs, f, indent=2)

print("✅ Finished: filtered domain sequences saved to autogen_filtered_domain_sequences.json")


In [ ]:
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
)

user_proxy = autogen.UserProxyAgent(
    name="user_proxy", 
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config={
        "work_dir": "web",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    llm_config=llm_config,
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
Otherwise, reply CONTINUE, or the reason why the task is not solved yet.""",
)

from datetime import datetime
from autogen.tools.experimental import DuckDuckGoSearchTool 

import requests

class DuckDuckGoWithUserAgent(DuckDuckGoSearchTool):
    def _get_html(self, url):
        headers = {
            "User-Agent": (
                "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                "AppleWebKit/537.36 (KHTML, like Gecko) "
                "Chrome/117.0.0.0 Safari/537.36"
            )
        }
        return requests.get(url, headers=headers, timeout=10)

wrapped_tool = ToolWrapperWithISPLogger(
    DuckDuckGoWithUserAgent(),
    base_url="https://html.duckduckgo.com/html/"
)

wrapped_tool.register_for_llm(assistant)
wrapped_tool.register_for_execution(user_proxy)



In [ ]:
# create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
)
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "web",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    llm_config=llm_config,
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
Otherwise, reply CONTINUE, or the reason why the task is not solved yet.""",
)
user_proxy.initiate_chat(
    assistant,
    message="""Compare recent advancements in LLM alignment techniques across OpenAI, Anthropic, and Meta. Provide key papers, blog posts, and implementations.""",
)

for msg in chat_result.chat_history:
    print(f"{msg['role']} ({msg.get('name', 'n/a')}):\n{msg.get('content', '[no content]')}\n")
